# 1. 네이버 맵 Geocoding API 이용

참고 : https://ring-wdr.github.io/python/2020/11/14/naver_map_api.html

In [1]:
import numpy as np
import pandas as pd
from urllib.request import urlopen
from urllib import parse
from urllib.request import Request
from urllib.error import HTTPError
from bs4 import BeautifulSoup
import json

## 1-1. 네이버 API ID, PW 불러오기  
`secrets.json`
```json
{
    "client_id":"id",
    "client_pw":"password"
}
```

In [4]:
secret_file = 'secrets.json'

# secrets.json 불러오기
with open(secret_file) as f:
    secrets = json.loads(f.read())

# secrets.json 파일에서 id, pw 불러오기
def get_secret(setting, secrets=secrets):
    try:
        return secrets[setting]
    except KeyError:
        error_msg = f'No such file(secrets.json) or key {setting}'
        raise ImproperlyConfigured(error_msg)
        
client_id = get_secret("client_id")
client_pw = get_secret("client_pw")

## 1-2. AED 주소 csv 파일 불러오기
- AED 주소 데이터 : [E-gen 자동심장충격기(AED) 찾기](https://www.e-gen.or.kr/egen/search_aed.do?searchType=general) - \[서울특별시\] - \[검색\] - \[검색결과 다운로드\]

In [5]:
data = pd.read_csv('./data/AED_Seoul.csv', engine='python', encoding='CP949')
data.head()

,NO,설치장소,전화번호,도로명주소,지번주소
0,1,(주)동아서울시네마 (극장 5층 들어가는 입구 벽면),02-745-4231,"서울특별시 종로구 삼일대로 428, 낙원상가 5층 (낙원동)",서울특별시 종로구 낙원동 288번지 낙원상가
1,2,(주)동아서울시네마 (극장 4층 사무실 옆 5층 올라가는 우측 계단 벽),02-745-4231,"서울특별시 종로구 삼일대로 428, 낙원상가 4층 (낙원동)",서울특별시 종로구 낙원동 288번지 낙원상가
2,3,(주)이마트가양점 (주차장동 3층),02-2101-1119,"서울특별시 강서구 양천로 559, 가양이마트 (가양동)",서울특별시 강서구 가양동 449번지 19호 가양이마트
3,4,(주)트랜스코스모스코리아 (12층 엘리베이터 홀),02-790-8971,"서울특별시 관악구 관악로 217, 동진빌딩 (봉천동)",서울특별시 관악구 봉천동 32번지 8호 동진빌딩
4,5,(주)트랜스코스모스코리아 (동진빌딩6층 엘리베이터홀),02-790-8971,"서울특별시 관악구 관악로 217, 동진빌딩 (봉천동)",서울특별시 관악구 봉천동 32번지 8호 동진빌딩


## 1-3. 주소 > 좌표로 변환

In [6]:
api_url = 'https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query='

In [9]:
# # 상위 5개 예시
# geo_coordi = []     
# for add in data.loc[:5, '지번주소']:
#     print(add)
#     add_urlenc = parse.quote(add)  
#     url = api_url + add_urlenc
#     request = Request(url)
#     request.add_header('X-NCP-APIGW-API-KEY-ID', client_id)
#     request.add_header('X-NCP-APIGW-API-KEY', client_pw)
#     try:
#         response = urlopen(request)
#     except:
#         print('HTTP Error')
#         latitude = None
#         longitude = None
#     else:
#         rescode = response.getcode()
#         if rescode == 200:
#             response_body = response.read().decode('utf-8')
#             response_body = json.loads(response_body)   # json
#             if response_body['addresses'] == [] :
#                 print('Cannot convert {add} to coordinate.')
#                 latitude = None
#                 longitude = None
#             else:
#                 latitude = response_body['addresses'][0]['y']
#                 longitude = response_body['addresses'][0]['x']
#         else:
#             print(f'Response error code : {rescode}')
#             latitude = None
#             longitude = None

#     geo_coordi.append([latitude, longitude])

# print(geo_coordi)

서울특별시 종로구 낙원동 288번지 낙원상가
서울특별시 종로구 낙원동 288번지 낙원상가
서울특별시 강서구 가양동 449번지 19호 가양이마트
서울특별시 관악구 봉천동 32번지 8호 동진빌딩
서울특별시 관악구 봉천동 32번지 8호 동진빌딩
서울특별시 관악구 봉천동 853번지 1호 호산빌딩
[['37.5722238', '126.9876399'], ['37.5722238', '126.9876399'], ['37.5595592', '126.8596072'], ['37.4680011', '126.9635703'], ['37.4680011', '126.9635703'], ['37.4805019', '126.9537464']]


In [ ]:
geo_coordi = []     
for add in data['지번주소']:
    add_urlenc = parse.quote(add)  
    url = api_url + add_urlenc
    request = Request(url)
    request.add_header('X-NCP-APIGW-API-KEY-ID', client_id)
    request.add_header('X-NCP-APIGW-API-KEY', client_pw)
    try:
        response = urlopen(request)
    except:
        print('HTTP Error')
        latitude = None
        longitude = None
    else:
        rescode = response.getcode()
        if rescode == 200:
            response_body = response.read().decode('utf-8')
            response_body = json.loads(response_body)   # json
            if response_body['addresses'] == [] :
                print('Cannot convert {add} to coordinate.')
                latitude = None
                longitude = None
            else:
                latitude = response_body['addresses'][0]['y']
                longitude = response_body['addresses'][0]['x']
        else:
            print(f'Response error code : {rescode}')
            latitude = None
            longitude = None

    geo_coordi.append([latitude, longitude])

In [11]:
# # 상위 5개 예시
# np_geo_coordi = np.array(geo_coordi, dtype=float)
# pd_geo_coordi = pd.DataFrame({"도로명": data.loc[:5, '도로명주소'].values,
#                               "지번주소": data.loc[:5, '지번주소'].values,
#                               "위도": np_geo_coordi[:, 0],
#                               "경도": np_geo_coordi[:, 1]})
# pd_geo_coordi.head()

,도로명,지번주소,위도,경도
0,"서울특별시 종로구 삼일대로 428, 낙원상가 5층 (낙원동)",서울특별시 종로구 낙원동 288번지 낙원상가,37.572224,126.987640
1,"서울특별시 종로구 삼일대로 428, 낙원상가 4층 (낙원동)",서울특별시 종로구 낙원동 288번지 낙원상가,37.572224,126.987640
2,"서울특별시 강서구 양천로 559, 가양이마트 (가양동)",서울특별시 강서구 가양동 449번지 19호 가양이마트,37.559559,126.859607
3,"서울특별시 관악구 관악로 217, 동진빌딩 (봉천동)",서울특별시 관악구 봉천동 32번지 8호 동진빌딩,37.468001,126.963570
4,"서울특별시 관악구 관악로 217, 동진빌딩 (봉천동)",서울특별시 관악구 봉천동 32번지 8호 동진빌딩,37.468001,126.963570


In [ ]:
np_geo_coordi = np.array(geo_coordi, dtype=float)
pd_geo_coordi = pd.DataFrame({"도로명": data['도로명주소'].values,
                              "지번주소": data['지번주소'].values,
                              "위도": np_geo_coordi[:, 0],
                              "경도": np_geo_coordi[:, 1]})
pd_geo_coordi.head()

이후 `AED_Seoul_coord.csv` 파일로 저장

In [ ]:
pd_geo_coordi.to_csv('AED_Seoul_coord.csv', sep=',', na_rep='NaN', encoding='CP949', index=None)

이후 좌표를 불러오지 못한 주소들은 구글 지도에 직접 찍어가며 결측값을 채움

## 1-4. 소방서 및 119 안전센터 주소 변환
- 서울시 소방서 주소 데이터 : [공공데이터포털 - 서울특별시 소방서 현황](https://www.data.go.kr/data/15047342/fileData.do)
- 서울시 119 안전센터 좌표 데이터 : [서울 열린데이터 광장 - 서울시 안전센터관할 위치정보](http://data.seoul.go.kr/dataList/OA-12740/S/1/datasetView.do)
- AED와 같은 방법으로 소방서 주소를 좌표로 변환한 후 `FS_Seoul_coord.csv` 파일로 저장
- 앞서 얻은 csv 파일과 119 안전센터 좌표 데이터를 엑셀을 이용하여 직접 합침

In [13]:
fs_df = pd.read_csv('./data/FS_Seoul_coord.csv', encoding='cp949')
fs_df.head()

,소방서명,위치,위도,경도
0,종로소방서,서울특별시 종로구 종로1길 28(수송동),37.572856,126.978913
1,중부소방서,서울특별시 중구 퇴계로 394(무학동),37.564867,127.015394
2,광진소방서,서울특별시 광진구 광나루로 480(구의동),37.544965,127.082812
3,용산소방서,서울특별시 용산구 한강대로 167(한강로2가),37.534122,126.971496
4,동대문소방서,서울특별시 동대문구 장한로 34(장안동),37.564004,127.066625


In [14]:
fs_df.tail()

,소방서명,위치,위도,경도
137,무학119안전센터,서울특별시 중구 퇴계로 394 (무학동),37.564867,127.015394
138,면목119안전센터,서울특별시 중랑구 동일로 475 (면목동),37.572857,127.079902
139,중화119안전센터,서울특별시 중랑구 동일로 783 (중화동),37.600459,127.079294
140,신내119안전센터,서울특별시 중랑구 신내로 183 (신내동),37.615321,127.094805
141,망우119안전센터,서울특별시 중랑구 용마산로 524 (망우동),37.597726,127.100572
